## using fixed_checked_flb to correct raw time-series

In [54]:
%matplotlib inline
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
prob_folder_path = '../../datasets/probability'
rever_prob_folder_path = '../../datasets/reverse_probability'
FLB_folder_path = '../../datasets/FLB'
hourly_FLB_folder_path = '../../datasets/FLB/hourly_FLB'
daily_FLB_folder_path = '../../datasets/FLB/daily_FLB'
global_FLB_folder_path = '../../datasets/FLB/global_FLB'
# train data
train_prob_folder_path = '../../datasets/train_data/probability'
corrected_prob_folder_path = '../../datasets/train_data/corrected_probability'
resample_prob_folder_path = '../../datasets/train_data/probability_resample'
resample_corrected_prob_folder_path = '../../datasets/train_data/corrected_probability_resample'
summary_folder_path = '../../datasets/train_data/summary'

games = { 'baseball':'/baseball/mlb','basketball':'/basketball/nba','football':'/football/nfl',
            'iceball': '/iceball/nhl','soccer_champion':'/soccer/champ_league','scoccer_england':'/soccer/epl',
            'soccer_major':'/soccer/majorleague'}
# folder name is games, result name is games_results.csv

games_list = ['baseball','iceball']
soccer_list = ['soccer_champion','scoccer_england','soccer_major']

In [78]:
"""
load FLB as maps

{'game1':{price:true_percentage,...},
'game2':{price:true_percentage,...}}
""" 

flb_maps = {}
for game_name in games_list:
    print(game_name)
    df = pd.read_csv(FLB_folder_path+'/'+game_name+'_fixed_checked.csv')[['price','perc']]
    flb_maps[game_name] = df.set_index('price').to_dict()['perc']
    
for game_name in ['soccer_home']:
    print(game_name)
    df = pd.read_csv(FLB_folder_path+'/'+game_name+'_fixed_checked.csv')[['price','perc']]
    flb_maps[game_name] = df.set_index('price').to_dict()['perc']

"\nload FLB as maps\n\n{'game1':{price:true_percentage,...},\n'game2':{price:true_percentage,...}}\n"

baseball
iceball
soccer_home


## correct using  flb maps

In [ ]:
#  other games
for game_name in games_list:
    print(game_name)
    index_list = [int(i[:-4]) for i in os.listdir(prob_folder_path + games[game_name])]
    for index in tqdm(index_list):
        series_df = pd.read_csv(prob_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('time')
        if len(series_df) < 2:
            continue
        series_df.index = pd.to_datetime(series_df.index)
        copy_df = series_df.copy()
        start = series_df.index[0]
        end  = series_df.index[-1]
        
#       no_resample :before correct
        copy_df['duration'] = [int((end - x).total_seconds()) for x in copy_df.index]
        copy_df.to_csv(train_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')
#       no_resample :after correct
        copy_df['home'] = copy_df['home'].apply(lambda x: round(flb_maps[game_name][round(x,2)],4))
        copy_df.to_csv(corrected_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')

#       resample:before correct
        series_df = series_df.reindex(pd.date_range(start, end, freq='10T'),method='nearest')
        series_df['duration'] = [int((end - x).total_seconds()) for x in series_df.index]
        series_df.to_csv(resample_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')
#       resample :after correct
        series_df['home'] = series_df['home'].apply(lambda x: round(flb_maps[game_name][round(x,2)],4))
        series_df.to_csv(resample_corrected_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')


In [81]:
# soccer
for game_name in soccer_list:
#     odds_type = 'home'
    print(game_name)
    index_list = [int(i[:-4]) for i in os.listdir(prob_folder_path + games[game_name]+'_normalized')]

    for index in tqdm(index_list):
        series_df = pd.read_csv(prob_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('time')
        if len(series_df) < 2:
            continue
        series_df.index = pd.to_datetime(series_df.index)
        copy_df = series_df.copy()
        start = series_df.index[0]
        end  = series_df.index[-1]
        
#       no_resample :before correct
        copy_df['duration'] = [int((end - x).total_seconds()) for x in copy_df.index]
        copy_df[['home','duration']].to_csv(train_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')
#       no_resample :after correct
        copy_df['home'] = copy_df['home'].apply(lambda x: round(flb_maps['soccer_home'][round(x,2)],4))
        copy_df[['home','duration']].to_csv(corrected_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')

#       resample:before correct
        series_df = series_df.reindex(pd.date_range(start, end, freq='10T'),method='nearest')
        series_df['duration'] = [int((end - x).total_seconds()) for x in series_df.index]
        series_df[['home','duration']].to_csv(resample_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')
#       resample :after correct
        series_df['home'] = series_df['home'].apply(lambda x: round(flb_maps['soccer_home'][round(x,2)],4))
        series_df[['home','duration']].to_csv(resample_corrected_prob_folder_path + games[game_name]+'/'+str(index)+'.csv')

soccer_champion



  1%|▉                                                                                 | 7/604 [00:00<00:52, 11.48it/s]
Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\surface\Anaconda2\envs\tensorflow\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\surface\Anaconda2\envs\tensorflow\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|████████████████████████████████████████████████████████████████████████████████| 604/604 [02:16<00:00,  4.41it/s]


scoccer_england


 16%|█████████████                                                                  | 322/1953 [01:

limit_output extension: Maximum message size of 100000 exceeded with 100020 characters

### daily & hourly correct

In [50]:
"""

daily

"""
# baseball
baseball_daily = pd.DataFrame()
game_name = 'baseball'
index_list = os.listdir(daily_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(daily_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    baseball_daily[inx] = df['perc']

# iceball
iceball_daily = pd.DataFrame()
game_name = 'iceball'
index_list = os.listdir(daily_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(daily_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    iceball_daily[inx] = df['perc']

# soccer
soccer_daily = pd.DataFrame()
game_name = 'soccer'
index_list = os.listdir(daily_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(daily_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    soccer_daily[inx] = df['perc']
    
"""   

hourly

"""
# baseball
baseball_hourly = pd.DataFrame()
game_name = 'baseball'
index_list = os.listdir(hourly_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(hourly_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    baseball_hourly[inx] = df['perc']

# iceball
iceball_hourly = pd.DataFrame()
game_name = 'iceball'
index_list = os.listdir(hourly_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(hourly_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    iceball_hourly[inx] = df['perc']

# soccer
soccer_hourly = pd.DataFrame()
game_name = 'soccer'
index_list = os.listdir(hourly_FLB_folder_path +  '/' + game_name+'_checked')
for index in index_list:
    df = pd.read_csv(hourly_FLB_folder_path +  '/' + game_name+'_checked/'+index).set_index('price')
    inx = int(index.split('_')[1].split('.')[0])
    soccer_hourly[inx] = df['perc']
    
""" 

global

"""       
baseball_global = pd.DataFrame()
game_name = 'baseball'
baseball_global['global'] = pd.read_csv(global_FLB_folder_path +  '/' + game_name+'_checked.csv').set_index('price')['perc']

iceball_global = pd.DataFrame()
game_name = 'iceball'
iceball_global['global'] = pd.read_csv(global_FLB_folder_path +  '/' + game_name+'_checked.csv').set_index('price')['perc']

soccer_global = pd.DataFrame()
game_name = 'soccer'
soccer_global['global'] = pd.read_csv(global_FLB_folder_path +  '/' + game_name+'_checked.csv').set_index('price')['perc']


In [36]:
def correct_by_time(time,home,flb):
#     join by dai and key
    if time > flb.columns.max():
        return flb.loc[round(home,2),flb.columns.max()]
    else:
        return flb.loc[round(home,2),time]

In [58]:
game_name = 'baseball'
index_list = [int(i[:-4]) for i in os.listdir(rever_prob_folder_path + games[game_name])]
for index in index_list:
    series_df = pd.read_csv(rever_prob_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('duration')

    series_df['daily_corrected'] = series_df.apply(lambda row: correct_by_time(row['day'], row['home'],baseball_daily), axis=1)
    series_df['hourly_corrected'] = series_df.apply(lambda row: correct_by_time(row['hour'], row['home'],baseball_hourly), axis=1)
    series_df['global_corrected'] = series_df.apply(lambda row: baseball_global.loc[round(row['home'],2),'global'], axis=1)
    series_df.to_csv(summary_folder_path + games[game_name] +'/'+str(index)+'.csv')

In [59]:
game_name = 'iceball'
index_list = [int(i[:-4]) for i in os.listdir(rever_prob_folder_path + games[game_name])]
for index in index_list:
    series_df = pd.read_csv(rever_prob_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('duration')

    series_df['daily_corrected'] = series_df.apply(lambda row: correct_by_time(row['day'], row['home'],iceball_daily), axis=1)
    series_df['hourly_corrected'] = series_df.apply(lambda row: correct_by_time(row['hour'], row['home'],iceball_hourly), axis=1)
    series_df['global_corrected'] = series_df.apply(lambda row: iceball_global.loc[round(row['home'],2),'global'], axis=1)
    series_df.to_csv(summary_folder_path + games[game_name] +'/'+str(index)+'.csv')

In [60]:
for game_name in soccer_list:
    print(game_name)
    index_list = [int(i[:-4]) for i in os.listdir(rever_prob_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(rever_prob_folder_path + games[game_name]+'/'+str(index)+'.csv').set_index('duration')

        series_df['daily_corrected'] = series_df.apply(lambda row: correct_by_time(row['day'], row['home'],soccer_daily), axis=1)
        series_df['hourly_corrected'] = series_df.apply(lambda row: correct_by_time(row['hour'], row['home'],soccer_hourly), axis=1)
        series_df['global_corrected'] = series_df.apply(lambda row: soccer_global.loc[round(row['home'],2),'global'], axis=1)
        series_df.to_csv(summary_folder_path + games[game_name] +'/'+str(index)+'.csv')

soccer_champion
scoccer_england
soccer_major
